## 로지스틱 회귀
* 가방 안에 어떤 물고기가 %정도 들어있는지 확률을 예측해보자
* 예측이지만 분류 모델인 로지스틱 회귀 사용
  * 이유는? 로지스틱 회귀는 출력을 0~1 사이로 출력하기 때문에 이를 확률로 이해할 수 있다


### 1.데이터준비

In [1]:
import pandas as pd

fish=pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
fish_input=fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_target=fish[['Species']].to_numpy()

### 2.데이터 분리

In [5]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target=train_test_split(fish_input, fish_target, random_state=42)

### 3.정규화 진행

In [6]:
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
ss.fit(train_input)
train_scaled=ss.transform(train_input)
test_scaled=ss.transform(test_input)

### 4.로지스틱회귀(다중회귀)
* 물고기의 X=길이, 넓이, 높이, 대각선, 무게
* 물고기의 y=물고기 종류 7가지

In [14]:
from sklearn.linear_model import LogisticRegression
import numpy as np

lr=LogisticRegression(C=20, max_iter=1000) #C는 규제를 제어하는 매개변수(c가 클수록 규제는 줄어듬), max_iter은 반복횟수
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))

0.9327731092436975


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
print(lr.score(test_scaled, test_target))

0.925


In [16]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [18]:
proba=lr.predict_proba(test_scaled[:5]) #처음 5개 샘플에 대한 예측 확률 출력
print(np.round(proba, decimals=3)) #소수점 네번째 자리에서 반올림

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [19]:
print(lr.classes_) #이 순서로 출력됨! 입력데이터 순서 아님!

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [21]:
 #선형 방정식은 어떻게 생겼을까?
 print(lr.coef_.shape, lr.intercept_.shape) #다중분류는 클래스마다 z값을 하나씩 계산한다.

(7, 5) (7,)


In [23]:
#저 확률은 어떻게 나왔을까?
#다중분류의 경우 z값을 소프트맥스 함수 적용해서 확률처럼 나옴
#함 해보자 똑같나 실험!!!!

decision=lr.decision_function(test_scaled[:5]) #decision_function() 메서드로 z1~z7까지의 값을 구해보자
print(np.round(decision, decimals=2))

#여기에 소프트맥스 함수 적용해보자
from scipy.special import softmax

proba=softmax(decision, axis=1)
print(np.round(proba, decimals=3))
#위에서 출력한 값과 똑같다!

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


## 정리
* 로지스틱 회귀를 사용해 7개의 생선에 대한 확률을 예측하는 모델을 훈련했다
* 학습을 통해 a,b,c,e (매개변수)를 최적으로 구하고 이를 통해 z값을 얻는다. 이 z값에 이진분류면 시그모이드, 다중분류면 소프트맥스를 적용해 결과를 출력한다